In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import operator

import peakutils

import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')

import pickle
from sklearn.feature_extraction.text import CountVectorizer

import plotly
import plotly.plotly as py
from plotly import tools

import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
# data_path = "/mnt/disks/vault/analysis-data/word-journey/all-data.pql"
# data = pd.read_pickle(data_path)

In [4]:
filename = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/all-data-grouped-by-label.pql"
pickle_in = open(filename,"rb")
data_grouped_by_label = pickle.load(pickle_in)

In [5]:
data_grouped_by_label.head()

,title_without_stopwords,title_words,title_words_
Label,,,
Acoustics,"[multiplexer using flat exponential filters, m...",multiplexer using flat exponential filters mul...,multiplexer using flat exponential filters mul...
Agricultural Economics & Policy,"[poultry eggs, accounting commodity credit cor...",poultry eggs accounting commodity credit corpo...,poultry eggs accounting commodity credit corpo...
Agricultural Engineering,"[soft energy from palm wastes, potential land ...",soft energy from palm wastes potential land di...,soft energy from palm wastes potential land di...
"Agriculture, Dairy & Animal Science",[collection 2 cell embryos their culture media...,collection cell embryos their culture media co...,collection 2 cell embryos their culture media ...
"Agriculture, Multidisciplinary",[response strawberry fruit yield plant populat...,response strawberry fruit yield plant populati...,response strawberry fruit yield plant populati...


In [6]:
vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(data_grouped_by_label.title_words)

In [7]:
vocab = {value:key for key,value in vectorizer.vocabulary_.items()}
count_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
words = [count_rev_dict[i] for i in range(len(vectorizer.vocabulary_.keys()))]

In [8]:
#rev_dict

In [9]:
vectorized_wordarray = vectorized_words.toarray()
document_frequency = (vectorized_wordarray != 0).sum(0).tolist()
document_frequency_log =  [np.log(x) for x in document_frequency]
#2. Term Frequency - Number of times in which a word appears in a document
term_frequency = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
term_frequency_log = [np.log(x) for x in term_frequency]

In [113]:
# what disciplines does the word appear in?

word_ = "actidion"

index = words.index(word_)
count_rev_dict[index]
counts = vectorized_wordarray[:,index]

discipline_indices = [discipline_index for discipline_index in range(len(counts.tolist())) if counts[discipline_index] != 0]

In [114]:
disciplines = [list(data_grouped_by_label.index)[index] for index in discipline_indices]

In [115]:
print(disciplines)
print(len(disciplines))

['Biochemistry & Molecular Biology', 'Biophysics', 'Chemistry, Organic', 'Infectious Diseases', 'Microbiology', 'Multidisciplinary Sciences']
6


In [116]:
filtered_data = []

for discipline in disciplines:
    filtered_data.append(data[data.Label == discipline])
    
#data = None    

frequencies = []

for data_ in tqdm(filtered_data):
    
    data_ = data_.sort_values(by="PubYear")
    vectorizer = CountVectorizer()
    vectorized_words = vectorizer.fit_transform(data_.title_without_stopwords)
    
    vocab = vectorizer.vocabulary_
    rev_vocab = {index:word for word,index in vocab.items()}
    
    row_counts = data_.groupby('PubYear').size().reset_index(name="count")
    counts =  list(row_counts["count"])
    
    word_index = vocab[word_]
    yearly_freq_for_word = []
    index_start = 0
    
    for i in range(len(row_counts)):
        
        offset = index_start + counts[i]
        tmp = vectorized_words[index_start:offset,:]
        yearly_word_freq = tmp.sum(axis=0).reshape(-1,).tolist()[0][word_index]
        
        ## Normalizing - Divide by number of physics articles in the year
        _tmp = float(yearly_word_freq) / counts[i]
        
        yearly_freq_for_word.append(_tmp)
        index_start = offset
#         
    frequencies.append(yearly_freq_for_word)
    

100%|██████████| 6/6 [02:03<00:00, 21.74s/it]


In [104]:
years = [sorted(_data.PubYear.unique()) for _data in filtered_data]

datatoplot = [go.Scatter(
            x=years[0],
            y=frequencies[0],
            mode='lines',
            name=disciplines[0],
    ),go.Scatter(
            x=years[1],
            y=frequencies[1],
            mode='lines',
            name=disciplines[1],
    ),go.Scatter(
            x=years[2],
            y=frequencies[2],
            mode='lines',
            name=disciplines[2],
    ),go.Scatter(
            x=years[3],
            y=frequencies[3],
            mode='lines',
            name=disciplines[3],
    ),go.Scatter(
            x=years[4],
            y=frequencies[4],
            mode='lines',
            name=disciplines[4],
    ),go.Scatter(
            x=years[5],
            y=frequencies[5],
            mode='lines',
            name=disciplines[5],
#     ),go.Scatter(
#             x=years[6],
#             y=frequencies[6],
#             mode='lines',
#             name=disciplines[6],
#     ),go.Scatter(
#             x=years[7],
#             y=frequencies[7],
#             mode='lines',
#             name=disciplines[7],
#     ),go.Scatter(
#             x=years[8],
#             y=frequencies[8],
#             mode='lines',
#             name=disciplines[8],
#     ),go.Scatter(
#             x=years[9],
#             y=frequencies[9],
#             mode='lines',
#             name=disciplines[9],
#     ),go.Scatter(
#             x=years[10],
#             y=frequencies[10],
#             mode='lines',
#             name=disciplines[10],
#     ),go.Scatter(
#             x=years[11],
#             y=frequencies[11],
#             mode='lines',
#             name=disciplines[11],
#     ),go.Scatter(
#             x=years[12],
#             y=frequencies[12],
#             mode='lines',
#             name=disciplines[12],
#     ),go.Scatter(
#             x=years[13],
#             y=frequencies[13],
#             mode='lines',
#             name=disciplines[13],
    )]



layout = go.Layout(
    title='Word Propagation ' + word_,
    
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Popularity Ratio',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    )
)



fig1 = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig1, filename='freq')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

